In [7]:
# dependencies
import pandas as pd

# name CSVs to import
school_csv = "Resources/schools_complete.csv"
student_csv = "Resources/students_complete.csv"

# read CSVs into pandas
school_df = pd.read_csv(school_csv)
student_df = pd.read_csv(student_csv)

In [17]:
# view raw school table
school_df.head(15)

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500
5,5,Wilson High School,Charter,2283,1319574
6,6,Cabrera High School,Charter,1858,1081356
7,7,Bailey High School,District,4976,3124928
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858


In [9]:
# view raw student table
student_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [10]:
# merge data into single dataset
merged_df = pd.merge(student_df, school_df, how="left", on=["school_name", "school_name"])
merged_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [11]:
merged_df.shape

(39170, 11)

## District Summary

In [12]:
# create array of unique school names
unq_school = merged_df['school_name'].unique()
# get the length of the array
school_count = len(unq_school)

# student count 
stu_count = student_df['student_name'].count()

# double check with district students
dist_students = school_df['size'].sum()

# total budget for all schools
total_budget = school_df['budget'].sum()

# average math score
avg_math = merged_df['math_score'].mean()

# average reading score
avg_reading = merged_df['reading_score'].mean()

# percent of students passing math (70 or greater)
math_num = merged_df.loc[merged_df['math_score']>= 70]['student_name'].count()
math_prcnt = math_num/stu_count*100

# percent of students passing reading (70 or greater)
reading_num = merged_df.loc[merged_df['reading_score']>= 70]['student_name'].count()
reading_prcnt = reading_num/stu_count*100

# percent of students with overall passing grades (70 or greater in both math & reading)
passing_num = merged_df.loc[(merged_df['math_score']>= 70) & (merged_df['reading_score'] >= 70)]['student_name'].count()
passing_prcnt = passing_num/stu_count*100


# display summary dataframe
district_summary = pd.DataFrame({
    "Total Schools": [school_count],
    "Total Students": [stu_count],
    "Total Budget": [total_budget],
    "Average Math Score": [avg_math],
    "Average Reading Score": [avg_reading],
    "% Passing Math": [math_prcnt],
    "% Passing Reading": [reading_prcnt],
    "% Overall Passing": [passing_prcnt]    
})

district_summary

# map format for ease of view
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,}".format)
district_summary["Average Math Score"] = district_summary["Average Math Score"].map("{:.2f}%".format)
district_summary["Average Reading Score"] = district_summary["Average Reading Score"].map("{:.2f}%".format)
district_summary["% Passing Math"] = district_summary["% Passing Math"].map("{:.2f}%".format)
district_summary["% Passing Reading"] = district_summary["% Passing Reading"].map("{:.2f}%".format)
district_summary["% Overall Passing"] = district_summary["% Overall Passing"].map("{:.2f}%".format)

district_summary


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428",78.99%,81.88%,74.98%,85.81%,65.17%


In [19]:
merged_df.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## School Summary

In [28]:
# use groupby to sort data into fields by school name and run calcluations
by_school = merged_df.set_index('school_name').groupby(['school_name'])
sch_type = school_df.set_index('school_name')['type']
stu_total = by_school['Student ID'].count()
sch_budget = school_df.set_index('school_name')['budget']
stu_budget = sch_budget/stu_total
sch_avg_math = by_school['math_score'].mean()
sch_avg_rdg = by_school['reading_score'].mean()
sch_math_prcnt = merged_df.loc[merged_df['math_score']>= 70].groupby('school_name')['Student ID'].count()/stu_total*100
sch_reading_prcnt = merged_df.loc[merged_df['reading_score']>= 70].groupby('school_name')['Student ID'].count()/stu_total*100
overall_prcnt = merged_df.loc[(merged_df['math_score']>= 70) & (merged_df['reading_score'] >= 70)].groupby('school_name')['Student ID'].count()/stu_total*100

# create dataframe to hold results
school_summary = pd.DataFrame({
    "School Type": sch_type,
    "Total Students": stu_total,
    "Total School Budget": sch_budget,
    "Per Student Budget": stu_budget,
    "Average Math Score": sch_avg_math,
    "Average Reading Score": sch_avg_rdg,
    "% Passing Math": sch_math_prcnt,
    "% Passing Reading": sch_reading_prcnt,
    "% Overall Passing": overall_prcnt
})

# map format for readability
school_summary["Total Students"] = school_summary["Total Students"].map("{:,}".format)
school_summary["Total School Budget"] = school_summary["Total School Budget"].map("${:,}".format)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map("${:.0f}".format)
school_summary["Average Math Score"] = school_summary["Average Math Score"].map("{:.2f}%".format)
school_summary["Average Reading Score"] = school_summary["Average Reading Score"].map("{:.2f}%".format)
school_summary["% Passing Math"] = school_summary["% Passing Math"].map("{:.2f}%".format)
school_summary["% Passing Reading"] = school_summary["% Passing Reading"].map("{:.2f}%".format)
school_summary["% Overall Passing"] = school_summary["% Overall Passing"].map("{:.2f}%".format)

school_summary


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,"4,976","$3,124,928",$628,77.05%,81.03%,66.68%,81.93%,54.64%
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06%,83.98%,94.13%,97.04%,91.33%
Figueroa High School,District,"2,949","$1,884,411",$639,76.71%,81.16%,65.99%,80.74%,53.20%
Ford High School,District,"2,739","$1,763,916",$644,77.10%,80.75%,68.31%,79.30%,54.29%
Griffin High School,Charter,"1,468","$917,500",$625,83.35%,83.82%,93.39%,97.14%,90.60%
Hernandez High School,District,"4,635","$3,022,020",$652,77.29%,80.93%,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087",$581,83.80%,83.81%,92.51%,96.25%,89.23%
Huang High School,District,"2,917","$1,910,635",$655,76.63%,81.18%,65.68%,81.32%,53.51%
Johnson High School,District,"4,761","$3,094,650",$650,77.07%,80.97%,66.06%,81.22%,53.54%
Pena High School,Charter,962,"$585,858",$609,83.84%,84.04%,94.59%,95.95%,90.54%


## Top Performing Schools (By % Overall Passing)

In [29]:
top_5 = school_summary.sort_values('% Overall Passing', ascending = False)
top_5.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06%,83.98%,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130",$638,83.42%,83.85%,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500",$625,83.35%,83.82%,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574",$578,83.27%,83.99%,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858",$609,83.84%,84.04%,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

In [30]:
bottom_5 = school_summary.sort_values('% Overall Passing', ascending = True)
bottom_5.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,"3,999","$2,547,363",$637,76.84%,80.74%,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411",$639,76.71%,81.16%,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635",$655,76.63%,81.18%,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020",$652,77.29%,80.93%,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650",$650,77.07%,80.97%,66.06%,81.22%,53.54%
